# This notebook can be used to query NIC for fast ice data
The goal is to use the functions herewith to query for fast ice data, and optionally to append the results to the existing version of the covariate grid.

In [1]:
## Loading the libraries
list.of.packages <- c("rgdal", "proj4","rgeos","maptools","raster","stringr","plyr","dplyr","xml2","httr","ggplot2","data.table")
lapply(list.of.packages, require, character.only = TRUE)


Loading required package: rgdal

Loading required package: sp

rgdal: version: 1.2-20, (SVN revision 725)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: /usr/share/gdal/2.2
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.4-1 

Loading required package: proj4


Attaching package: ‘proj4’


The following object is masked from ‘package:rgdal’:

    project


Loading required package: rgeos

rgeos version: 0.5-2, (SVN revision 621)
 GEOS runtime version: 3.6.2-CAPI-1.10.2 
 Linking to sp version: 1.4-1 
 Polygon checking: TRUE 


Loading required package: maptools

Checking rgeos availability: TRUE

Loading required package: raster

Loading required package: stringr

Loading required package: plyr

Loading required package: dplyr


Attaching package: ‘dplyr

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

In [2]:
## Setting needed directories
pathToGit<-"/home/ubuntu/fastIce/FastIceCovars/"
nicsavedir<-"/home/ubuntu/fastIce/nictemp/"
nicresultsdir<-"/home/ubuntu/fastIce/nicresults/"
